# Parte 1: Entrenamiento, validación y selección


In [1]:
# Importamos las librerias que utilizaremos a lo largo del proyecto
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from sklearn import svm
from sklearn import metrics
from sklearn import tree
import sklearn
import pandas as pd
import os
from datetime import datetime
import math

C:\Users\Chofo\Anaconda3\envs\galileo_python\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Train-val-test-split

El primer es dar un vistazo a los datos con los que se cuenta. Iniciamos validando el tipo de dato de cada uno de ellos.

In [2]:
# Iniciamos leyendo el archivo de entrada
dataset = pd.read_csv("data_titanic_proyecto.csv")

# Contamos el número de regisitros y el numero de columnas que tiene el dataset
print(dataset.shape)

(891, 12)


In [6]:
# Vemos el tipo de dato que tiene cada columna en el dataset
dataset.dtypes

PassengerId             int64
Name                   object
Age                   float64
SibSp                   int64
Parch                   int64
Ticket                 object
Fare                  float64
Cabin                  object
Embarked               object
passenger_class        object
passenger_sex          object
passenger_survived     object
dtype: object

In [7]:
# Calculamos el portancaje de NaN en nuestro dataset
pd.DataFrame({'Porcentaje NaN': dataset.isnull().sum() * 100 / len(dataset)})

,Porcentaje NaN
PassengerId,0.000000
Name,0.000000
Age,19.865320
SibSp,0.000000
Parch,0.000000
Ticket,0.000000
Fare,0.000000
Cabin,77.104377
Embarked,0.224467
passenger_class,0.000000


Con base al análisis anterior, procedemos a descartar los features de: PassengerId, Name, Ticket y Cabin.

La columna 'Cabin' se descarta debido a que posee un 77% de datos NaN los cuales representan una gran incertidumbre. A continuación procedemos a definir 3 variables, una que contendrá las features que no se utilizarán, otra que contendrá las features categoricas y la última el nombre del feature a predecir, en este caso será passenger_survived.

In [3]:
def feature_engineering(ds, useless_features, categorical_features):
    
    # Eliminamos las features que no se utilizaran del dataset
    ds = ds.drop(useless_features, axis=1)
    
    # Normalizamos las variables que no son categoricas
    # Sustituiremos los valores NaN por el valor de la media
    imputer = Imputer(strategy="median")
    ds_numeric_features = ds.drop(categorical_features, axis=1)
    imputer.fit(ds_numeric_features)
    ds_imputer = imputer.transform(ds_numeric_features)
    ds_numeric_features = pd.DataFrame(ds_imputer, columns=ds_numeric_features.columns)
    
    # Normalizamos las variables categoricas
    ds_categorical_features = ds.drop(ds_numeric_features.columns, axis=1)
    ds_categorical_features_encoded = pd.DataFrame()
    
    # Sustituiremos los valores NaN por 'NaN'
    ds_categorical_features = ds_categorical_features.replace(np.nan, 'NaN', regex=True)
    
    # Usamos LabelEncoder para convertir nuestros valores categoricos a numericos
    ds_categorical_features = ds_categorical_features.apply(LabelEncoder().fit_transform)
    ds_categorical_features = pd.DataFrame(ds_categorical_features, columns= ds_categorical_features.columns)
    
    # Concatenamos nuestros dos DataFrames
    ds[categorical_features] = ds_categorical_features[categorical_features].values
    ds[ds_numeric_features.columns] = ds_numeric_features.values
    
    return ds

In [4]:
# Aplicamos feature engineering para normalizar los datos a una forma que no sea útil
useless_features = np.array(['PassengerId','Name','Ticket', 'Cabin'])
categorical_features = np.array(['passenger_sex','Embarked','passenger_class','passenger_survived'])
normalized_dataset = feature_engineering(dataset, useless_features, categorical_features)

print (normalized_dataset)

      Age  SibSp  Parch      Fare  Embarked  passenger_class  passenger_sex  \
0    22.0    1.0    0.0    7.2500         3                0              1   
1    38.0    1.0    0.0   71.2833         0                2              0   
2    26.0    0.0    0.0    7.9250         3                0              0   
3    35.0    1.0    0.0   53.1000         3                2              0   
4    35.0    0.0    0.0    8.0500         3                0              1   
5    28.0    0.0    0.0    8.4583         2                0              1   
6    54.0    0.0    0.0   51.8625         3                2              1   
7     2.0    3.0    1.0   21.0750         3                0              1   
8    27.0    0.0    2.0   11.1333         3                0              0   
9    14.0    1.0    0.0   30.0708         0                1              0   
10    4.0    1.0    1.0   16.7000         3                0              0   
11   58.0    0.0    0.0   26.5500         3         

C:\Users\Chofo\Anaconda3\envs\galileo_python\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


Luego que hemos hecho feature_engineering sobre nuestra data para normalizarla, procedemos a crear nuestros distintos datasets que utilizaremos para entrenar, validar y probar.

In [5]:
# Hacemos una copia de nuestro dataset normalizado sin incluir la variable a predecir
ds_x = normalized_dataset.drop('passenger_survived', axis=1)

predict_feature = 'passenger_survived'
ds_y = normalized_dataset[predict_feature].copy()

# Separamos nuestros datos en datos de prueba y datos de entrenamiento
# Utiliazaremos el 80% como datos de entrenamiento
x_train, x_test, y_train, y_test = train_test_split(ds_x, ds_y, train_size=0.8)

# Utilizamos los datos de entrenamiento para extraer una porción para datos de validación.
# Utilizaremos un 20% del dataset de entrenamiento para datos de validación
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, train_size=0.85)

print('Shape de los datos de entrenamiento: ', x_train.shape, y_train.shape)
print('Shape de los datos de validación', x_validation.shape, y_validation.shape)
print('Shape de los datos de prueba', x_test.shape, y_test.shape)

Shape de los datos de entrenamiento:  (605, 7) (605,)
Shape de los datos de validación (107, 7) (107,)
Shape de los datos de prueba (179, 7) (179,)


# Registro de bitácora

Definimos unos métodos que serán de utilidad para registrar la información relevante de cada experimento que se haga con cada uno de los modelos.

In [68]:
def registrar_experimento(df, nombre_archivo):
    if not os.path.isfile(nombre_archivo):
        # Si el archivo no existe, lo crearmos y agregamos el dataframe con todo y sus encabezados
        df.to_csv(nombre_archivo)
    else: 
        # Si el archivo ya existe, agregamos el dataframe sin incluir sus encabezados
        df.to_csv(nombre_archivo, mode='a', header=False)

# Ensemble learning

A lo largo del proyecto se utilizará Ensemble Learning de cuatro distintos modelos obtenidos a partir de:

- Árbol de decisión con sklearn
- SVM con sklearn
- Naive bayes con numpy y/o pandas
- Reg. logística binaria(sigmoid) utilizando Tensorflow


# Bootstraping

A pesar de que en el proyecto no se utiliza muestreo por bootstrap, numpy provee de la función "numpy.random.sample" en donde debería colocarse el parámetro replace = True.

Básicamente lo que hace es tomar muestras aleatorias del dataset que se tiene. Esto es de gran utilidad para evitar que todos los datos para test, train y validation se encuentren ubicados en las mismas secciones del dataset. Esto hará que cada uno de éstos sets de datos esté conformado de valores aleatorios dentro del dataset.

# Support Vector Machine (SVM)

In [7]:
# Definimos función para crear modelo basado en SVM
# Este recibe los hiperparámetros de kernel, c (regularización) y gamma
def svm_model(x_train, y_train, x_test, y_test, kernel, c, gamma):
    # Registramos la hora de inicio
    start_time = datetime.now()
    
    # Ejecutamos SVM
    classifier = svm.SVC(kernel=kernel, C=c, gamma=gamma)
    classifier.fit(x_train, y_train)
    y_predict = classifier.predict(x_test)
    
    # Registramos la hora de finalización
    end_time = datetime.now()
    
    # Calculamos algunas métricas del modelo
    model_score = classifier.score(x_test, y_test)
    model_accuracy = metrics.accuracy_score(y_test, y_predict)
    model_precision = metrics.precision_score(y_test, y_predict)
    model_recall = metrics.recall_score(y_test, y_predict)
    
    # Creamos un DataFrame con la información a almacenar en csv
    data = {'fecha_hora_inicio':[start_time.strftime("%d/%m/%Y, %H:%M:%S")],
               'fecha_hora_fin':[end_time.strftime("%d/%m/%Y, %H:%M:%S")],
               'kernel':[kernel],
               'c':[c],
               'gamma':[gamma],
               'model_score':[model_score],
               'model_accuracy':[model_accuracy],
               'model_precision':[model_precision],
               'model_recall':[model_recall]}
    
    df = pd.DataFrame(data)
    registrar_experimento(df, 'svm_output.csv')
    joblib.dump(classifier, 'svm_clasificador.pkl')
    return df
    
    # ESTRUCTURA DEL DF DE RESULTADO 
    # fecha_hora_inicio: Fecha y hora de inicio
    # fecha_hora_fin: Fecha y hora de finalizacion
    # kernel: Kernel usado
    # c: Valor de regularización (c)
    # gamma: Valor de gamma
    # model_score: Punteo del SVM
    # model_accuracy: Accuracy del SVM
    # model_precision: Precision del SVM
    # model_recall: Recall del SVM    

In [67]:
# Aplicamos SVM a nuestros datos de entrenamiento y validación
# Variamos los valores de C y gamma para obtener distintos resultados y de éstos se elegirá el mejor
c = 1
ds_resultados_svm = pd.DataFrame()

while c > 0:
    gamma = 1
    while gamma > 0:
        model_svm = svm_model(x_train, y_train, x_validation, y_validation, 'linear', c, gamma)
        ds_resultados_svm = ds_resultados_svm.append(model_svm)
        
        if gamma > 10:
            gamma = float('%.4f'%(gamma - 10))
        elif gamma > 1:
            gamma = float('%.4f'%(gamma - 1))
        #elif gamma > 0.1:
            #gamma = float('%.4f'%(gamma - 0.1))
        #elif gamma > 0.01:
            #gamma = float('%.4f'%(gamma - 0.01))
        #elif gamma > 0.001:
            #gamma = float('%.4f'%(gamma - 0.001))
        else:
            gamma = 0
    if c > 10:
        c = float('%.4f'%(c - 10))
    elif c > 1:
        c = float('%.4f'%(c - 1))
    #elif c > 0.1:
        #c = float('%.4f'%(c - 0.1))
    #elif c > 0.01:
        #c = float('%.4f'%(c - 0.01))
    #elif c > 0.001:
        #c = float('%.4f'%(c - 0.001))
    else:
        c = 0

# Imprimimos los resultados obtenidos
print(ds_resultados_svm)

# Imprimimos el modelo con el mayor score
idxmax = ds_resultados_svm['model_score'].idxmax()
print(ds_resultados_svm.loc[idxmax])

      fecha_hora_inicio        fecha_hora_fin  kernel  c  gamma  model_score  \
0  14/07/2019, 22:37:46  14/07/2019, 22:38:06  linear  1      1     0.813084   

   model_accuracy  model_precision  model_recall  
0        0.813084         0.806452      0.641026  
fecha_hora_inicio    14/07/2019, 22:37:46
fecha_hora_fin       14/07/2019, 22:38:06
kernel                             linear
c                                       1
gamma                                   1
model_score                      0.813084
model_accuracy                   0.813084
model_precision                  0.806452
model_recall                     0.641026
Name: 0, dtype: object


# Árbol de decisión

In [8]:
# https://scikit-learn.org/stable/modules/tree.html
# Definimos función para crear modelo basado en Decision Tree
def decision_tree_model(x_train, y_train, x_test, y_test):
    # Registramos la hora de inicio
    start_time = datetime.now()
    
    # Ejecutamos Decision Tree
    classifier = tree.DecisionTreeClassifier()
    classifier.fit(x_train, y_train)
    y_predict = classifier.predict(x_test)
    
    # Registramos la hora de finalización
    end_time = datetime.now()
    
    # Calculamos algunas métricas del modelo
    model_score = classifier.score(x_test, y_test)
    model_accuracy = metrics.accuracy_score(y_test, y_predict)
    model_precision = metrics.precision_score(y_test, y_predict)
    model_recall = metrics.recall_score(y_test, y_predict)
    
    # Creamos un DataFrame con la información a almacenar en csv
    data = {'fecha_hora_inicio':[start_time.strftime("%d/%m/%Y, %H:%M:%S")],
               'fecha_hora_fin':[end_time.strftime("%d/%m/%Y, %H:%M:%S")],
               'model_score':[model_score],
               'model_accuracy':[model_accuracy],
               'model_precision':[model_precision],
               'model_recall':[model_recall]}
    
    df = pd.DataFrame(data)
    registrar_experimento(df, 'decision_tree_output.csv')
    joblib.dump(classifier, 'decision_tree_clasificador.pkl')
    return df
    
    # ESTRUCTURA DEL DF DE RESULTADO 
    # fecha_hora_inicio: Fecha y hora de inicio
    # fecha_hora_fin: Fecha y hora de finalizacion
    # model_score: Punteo del Decision Tree
    # model_accuracy: Accuracy del Decision Tree
    # model_precision: Precision del Decision Tree
    # model_recall: Recall del Decision Tree

In [137]:
# Aplicamos Decision Tree a nuestros datos de entrenamiento y validación
ds_resultados_decision_tree = decision_tree_model(x_train, y_train, x_validation, y_validation)
print(ds_resultados_decision_tree)

      fecha_hora_inicio        fecha_hora_fin  model_score  model_accuracy  \
0  13/07/2019, 15:10:39  13/07/2019, 15:10:39     0.719626        0.719626   

   model_precision  model_recall  
0         0.612903      0.513514  


# Naive Bayes

In [33]:
def naive_bayes_summary(x_train, y_train, y_column_name, y_value):
    # Calculamos los valores para Y
    summary = pd.DataFrame(columns=['Column', 'Item', 'Probability'])
    summary_conditional = pd.DataFrame(columns=['Column', 'YItem', 'Item', 'Probability'])
    y_total_rows = y_train.shape[0]
    y_column_distinct_values = y_train.unique().tolist()
    for column_value in y_column_distinct_values:
        value_probability = y_train[y_train == column_value].count() / y_total_rows
        summary = summary.append({'Column': y_column_name, 'Item': column_value, 'Probability': value_probability}, ignore_index=True)
    
    # Construimos nuestras tablas de probabilidad utilizando la data en x_train y y_train
    x_total_rows = x_train.shape[0]
    x_colnames = x_train.columns
    for column in x_colnames:
        column_distinct_values = x_train[column].unique().tolist()
        for column_value in column_distinct_values:
            column_values = x_train[column]
            value_probability = column_values[column_values == column_value].count() / x_total_rows
            summary = summary.append({'Column': column, 'Item': column_value, 'Probability': value_probability}, ignore_index=True)
            for y_column_value in y_column_distinct_values:
                combined_columns = pd.DataFrame(columns=['YColumn', 'XColumn'])
                combined_columns['YColumn'] = y_train
                combined_columns['XColumn'] = column_values
                combined_columns = combined_columns[combined_columns['YColumn']==y_column_value]
                y_column_value_count = len(combined_columns)
                combined_columns = combined_columns[combined_columns['XColumn']==column_value]
                combined_columns_count = len(combined_columns)
                summary_conditional = summary_conditional.append({'Column': column, 'YItem': y_column_value, 'Item': column_value, 'Probability': combined_columns_count/y_column_value_count}, ignore_index=True)
    
    # Calculamos nuestra tabla final
    numerador = 1
    denominador = 1
    
    # Calculamos el total de registros de y_train
    y_train_count = len(y_train)
    y_value_count = len(y_train[y_train==y_value])
    y_value_probability = y_value_count / y_train_count
    
    x_colnames = x_train.columns
    naive_bayes_summary = pd.DataFrame(columns=['Column', 'Item', 'Probability'])
    for column in x_colnames:
        column_distinct_values = x_train[column].unique().tolist()
        for column_value in column_distinct_values:
            conditional_probability = summary_conditional[(summary_conditional['YItem']==y_value) 
                                                          & (summary_conditional['Column']==column)
                                                         & (summary_conditional['Item']==column_value)]
            
            probability_x = summary[(summary['Column']==column) 
                                    & (summary['Item']==column_value)]
            
            numerador = conditional_probability['Probability'].iloc[0] * y_value_probability
            denominador = probability_x['Probability'].iloc[0]
            naive_bayes_summary = naive_bayes_summary.append({'Column': column, 'Item': column_value, 'Probability': numerador/denominador}, ignore_index=True)
    
    return naive_bayes_summary

In [40]:
# Se envía el valor de 1 en el valor de y que se quiere predecir, es decir, la probabilidad de que y tome ese valor
# En este caso, y = 1, que implica que la persona se salva
# Tomaremos en cuenta unicamente las variables categoricas
x_label_categoricas = np.delete(categorical_features, np.where(categorical_features == predict_feature), axis=0)
x_train_categoricas = x_train[x_label_categoricas]
summary = naive_bayes_summary(x_train_categoricas, y_train, predict_feature, 1)
print(summary)

            Column Item  Probability
0    passenger_sex    0     0.755869
1    passenger_sex    1     0.181122
2         Embarked    0     0.575221
3         Embarked    2     0.358491
4         Embarked    3     0.334096
5         Embarked    1     1.000000
6  passenger_class    2     0.629870
7  passenger_class    0     0.236527
8  passenger_class    1     0.478632


In [102]:
# Calculamos la probabilidad de sobrevivir para los items en el set de validación
x_validation_categoricas = x_validation[x_label_categoricas]
for index, row in x_validation_categoricas.iterrows():
    probabilidad = 1;
    for column in row.index:
        probabilidad_column = summary[(summary['Column']==column) 
                               & (summary['Item']==row[column])]
        probabilidad = probabilidad * probabilidad_column['Probability'].values[0]
    print('Características: ')
    print(row.index)
    print('Valores')
    print(row.values)
    print('Probabilidad de ',predict_feature, ' = ',1)
    print(probabilidad)

Características: 
Index(['passenger_sex', 'Embarked', 'passenger_class'], dtype='object')
Valores
[1 3 0]
Probabilidad de  passenger_survived  =  1
0.014312790847576787
Características: 
Index(['passenger_sex', 'Embarked', 'passenger_class'], dtype='object')
Valores
[1 3 1]
Probabilidad de  passenger_survived  =  1
0.0289631548212784
Características: 
Index(['passenger_sex', 'Embarked', 'passenger_class'], dtype='object')
Valores
[1 3 1]
Probabilidad de  passenger_survived  =  1
0.0289631548212784
Características: 
Index(['passenger_sex', 'Embarked', 'passenger_class'], dtype='object')
Valores
[0 3 1]
Probabilidad de  passenger_survived  =  1
0.12087037142633884
Características: 
Index(['passenger_sex', 'Embarked', 'passenger_class'], dtype='object')
Valores
[0 3 2]
Probabilidad de  passenger_survived  =  1
0.15906282993013907
Características: 
Index(['passenger_sex', 'Embarked', 'passenger_class'], dtype='object')
Valores
[0 2 0]
Probabilidad de  passenger_survived  =  1
0.06409211871

[0 3 1]
Probabilidad de  passenger_survived  =  1
0.12087037142633884
Características: 
Index(['passenger_sex', 'Embarked', 'passenger_class'], dtype='object')
Valores
[0 3 0]
Probabilidad de  passenger_survived  =  1
0.059730797855734055
Características: 
Index(['passenger_sex', 'Embarked', 'passenger_class'], dtype='object')
Valores
[1 0 2]
Probabilidad de  passenger_survived  =  1
0.0656233215042137
Características: 
Index(['passenger_sex', 'Embarked', 'passenger_class'], dtype='object')
Valores
[1 3 2]
Probabilidad de  passenger_survived  =  1
0.03811489379251955
Características: 
Index(['passenger_sex', 'Embarked', 'passenger_class'], dtype='object')
Valores
[0 3 0]
Probabilidad de  passenger_survived  =  1
0.059730797855734055
Características: 
Index(['passenger_sex', 'Embarked', 'passenger_class'], dtype='object')
Valores
[1 2 1]
Probabilidad de  passenger_survived  =  1
0.031077936738314105
Características: 
Index(['passenger_sex', 'Embarked', 'passenger_class'], dtype='object'

# Regresión Logística

In [ ]:
def regresion_logistica(x, y, labels, epochs, batch_size, lr, imprimir_error_cada):
    
    # Preparamos los valores de x a como los utilizaremos
    # Convertimos las dimensiones 28 X 28 en una dimension de 784
    x = np.reshape(x, x.shape[:-2] + (-1,))
    
    # Calculamos algunos valores a utilizar en la creación de los placeholders y variables
    n_parametros = np.prod(x[:1:2].shape)
    n_categorias = len(labels)
    n_items = x.shape[0]
    
    # Utilizamos one hot encoding para definir los valores de los labels y crear un arreglo con los valores del one hot encoding
    labels_values = np.array(list(labels.values()))
    sklb = sklearn.preprocessing.LabelBinarizer()
    sklb.fit(labels_values)
    y = sklb.transform(y)
    
    # Creamos el grafo que utilizaremos
    grafo = tf.Graph()
    with grafo.as_default():
        
        # Definimos nuestros placeholders
        x_ = tf.placeholder(tf.float32, [None,n_parametros], name="x")
        y_ = tf.placeholder(tf.float32, [n_categorias, None], name="y")
        lr_ = tf.placeholder(tf.float32, name="learning_rate")
        
        # Definimos nuestras variables
        w = tf.Variable(np.zeros([n_parametros,n_categorias]), name="w", dtype= tf.float32) 
        b = tf.Variable(np.ones([1, n_categorias]), name="b", dtype=tf.float32)
        
        # Función para calcular el valor de la hipotesis y
        with tf.name_scope("hipotesis"):
            yhat = tf.nn.softmax(tf.matmul(x_, w, name="matmul_xw") + b, name="yhat")
            
        # Función de costo con entropía cruzada
        with tf.name_scope("costo_entropia_cruzada"):
            cross_entropy = -1 * tf.reduce_mean(
                tf.matmul(y_, tf.log(yhat, name="log_yhat"), name="matmul_y_logyhat") 
                + tf.matmul((1 - y_),tf.log(1 - yhat, name="log_uno_menos_yhat"), name="matmul_uno_menos_y_por_log_uno_menos_yhat")
                , reduction_indices=[1], name="entropia_cruzada")
            #cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels = y_, logits = yhat)
            
        # Definimos el optimizador gradient descent
        with tf.name_scope("optimizador_gradient_descent"):
            optimizador = tf.train.GradientDescentOptimizer(lr_).minimize(cross_entropy) 
            
        # Calculamos el valor correcto de la predicción
        with tf.name_scope("prediccion_correcta"):
            prediccion_correcta = tf.equal(tf.argmax(y_,axis=0), tf.argmax(yhat,axis=1), name="prediccion_correcta")
            
        # Calculamos que tan acertado ha estado el modelo
        with tf.name_scope("exactitud"):
            exactitud = tf.reduce_mean(tf.cast(prediccion_correcta, tf.float32), name="exactitud")
        
        # Inicializamos las variables globales
        init = tf.global_variables_initializer()
        
        # Inicializamos el grafo
        with tf.Session(graph = grafo) as session:
            
            # Creamos un writer para nuestros logs en tensorboard
            writer = tf.summary.FileWriter('./grafos/lr=' + str(lr), session.graph)
            
            # Inicializamos las variables en la sesión
            session.run(init)
            
            # Iteramos el número de veces que indica el parámetro "epochs"
            for epoch in range(epochs):
                
                # Iteramos de acuerdo al numero de mini batches a utilizar
                for i in range(0, n_items, batch_size):
                    x_batch = x[i: i + batch_size,:]
                    y_batch = y[i: i + batch_size]
                    
                    # y_batch tiene la forma [batch_size, n_parametros]
                    # cambiamos y_batch a la forma [n_parametros, batch_size]
                    y_batch = np.transpose(y_batch)

                    # Ejecutamos el grafo
                    session.run(optimizador, feed_dict={x_:x_batch, y_:y_batch, lr_:lr})
                    
                    #if (epoch + 1) % imprimir_error_cada == 0:
                    #    costo = session.run(cross_entropy, feed_dict={x_:x_batch, y_:y_batch, lr_:lr})
                    #    print('Iteración: ' + str(epoch + 1))
                    #    print('Entropía cruzada: ')
                    #    print(costo)
                
                if (epoch + 1) % imprimir_error_cada == 0:
                    y_t = np.transpose(y)
                    resultado_exactitud = session.run(exactitud, feed_dict={x_:x, y_:y_t})
                    costo = session.run(cross_entropy, feed_dict={x_:x, y_:y_t})
                    print('epoch: ', epoch + 1)
                    print('Entropía cruzada: ', costo)
                    print('exactitud: ', resultado_exactitud)
            
                y_t = np.transpose(y)
                costos = session.run(cross_entropy, feed_dict={x_:x, y_:y_t})
                weights = session.run(w, feed_dict={x_:x, y_:y_t})
                bias = session.run(b, feed_dict={x_:x, y_:y_t})
            
            
            print('RESULTADOS FINALES')
            print(costos)
            print(weights)
            print(bias)
            
            # Cerramos el writer
            writer.close()